In [98]:
import pandas as pd
import streamlit as st
import sqlite3
import pydeck as pdk

In [106]:
# Connexion à la base de données
connexion = sqlite3.connect('bdd.db')

# Récupération des données de la table objet
query_objet = "SELECT * FROM objet"
df_objet = pd.read_sql_query(query_objet, connexion)

# Récupération des données de la table gare
query_gare = "SELECT * FROM gare"
df_gare = pd.read_sql_query(query_gare, connexion)

# Fermeture de la connexion à la base de données
connexion.close()

# Sélection de l'année et du type d'objet avec des widgets Streamlit
annee = '2020'

types = df_objet['type'].unique().tolist()
types_selectionnes = st.multiselect("Sélectionnez le type d'objet :", types, default=types[:], key=3)

# Filtrage des données en fonction de la sélection de l'utilisateur
df_objet_filtre = df_objet[(df_objet['type'].isin(types_selectionnes)) & (df_objet['date'].str[-4:] == annee)]
df_gare_filtre = df_gare[['gare', 'lat', 'lon', 'frequentation_' + annee]]

# Calcul du nombre d'objets trouvés des types sélectionnés pour chaque gare
df_nb_objets_par_gare = df_objet_filtre.groupby('gare')['id'].count().reset_index().rename(columns={'id': 'nb_objets'})
df_nb_objets_par_gare

,gare,nb_objets
0,Paris Austerlitz,239
1,Paris Bercy,325
2,Paris Est,844
3,Paris Gare de Lyon,4358
4,Paris Gare du Nord,2101
5,Paris Montparnasse,4113
6,Paris Saint-Lazare,1595


In [113]:
def load_data():
    conn = sqlite3.connect("bdd.db")
    query = """
    SELECT objet.date, objet.gare, objet.type, objet.nature, gare.lat, gare.lon, temperature.temperature
    FROM objet
    JOIN gare ON objet.gare = gare.gare
    JOIN temperature ON objet.date = temperature.date
    """
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def prepare_data(df):
    df['nombre_objets'] = 1
    df_filtre = df.groupby(['gare', 'temperature'])['nombre_objets'].sum().reset_index()
    return df_filtre

st.title("Analyse des objets trouvés en fonction de la température")

df = load_data()
df_filtre = prepare_data(df)
df_filtre

,gare,temperature,nombre_objets
0,Paris Austerlitz,-3.5,1
1,Paris Austerlitz,-2.8,2
2,Paris Austerlitz,-2.6,1
3,Paris Austerlitz,-2.3,4
4,Paris Austerlitz,-2.0,8
...,...,...,...
1862,Paris Saint-Lazare,28.8,11
1863,Paris Saint-Lazare,29.3,5
1864,Paris Saint-Lazare,30.4,8
1865,Paris Saint-Lazare,30.7,15


In [2]:
import colorcet as cc

def plasma_color(value, min_value, max_value):
    value_normalized = (value - min_value) / (max_value - min_value)
    color_index = int(value_normalized * 255)
    return list(cc.palette["CET_L8"][color_index])

plasma_color(100,100,10)

['#', '0', '0', '0', 'e', '5', 'c']